In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import scipy.stats as stats
# from scipy.stats import linregress
# from scipy.stats.stats import pearsonr
import warnings
%matplotlib inline

In [2]:
# Read-in data as Excel file
vix_data_file = "C:/Users/blake/Desktop/spx_vix_clean.xlsx"

In [3]:
#Convert Excel file to Dataframe
stock_data_df = pd.read_excel(vix_data_file,
sheet_name=0,
header=1,
index_col=False,
keep_default_na=False
)

In [4]:
stock_data_df.head()

,Date,SPX,VIX_1,VIX_2,VIX_3,VIX_4,VIX_5,VIX_6,VIX_7,VIX_8,stdev,1/stdev,SPX_VOLUME,SPX_5day,SPX_tom,A_Delta
0,2009-12-31,1115.10,22.95,25.80,26.00,26.30,26.35,26.30,26.40,26.35,1.172737,0.852706,636905,NaN,1132.98,17.88
1,2010-01-01,1132.98,22.10,24.85,25.25,25.75,25.75,25.75,25.85,25.95,1.287977,0.776412,636905,NaN,1132.98,0.00
2,2010-01-04,1132.98,22.10,24.85,25.25,25.75,25.75,25.75,25.85,25.95,1.287977,0.776412,1282633,NaN,1136.52,3.54
3,2010-01-05,1136.52,21.55,24.45,24.85,25.45,25.50,25.50,25.55,25.70,1.386655,0.721160,1368386,NaN,1137.14,0.62
4,2010-01-06,1137.14,20.80,23.55,24.20,24.95,25.05,25.05,25.10,25.30,1.511385,0.661645,1252015,1130.944,1141.69,4.55


In [5]:
#Create new Dataframe, set 'Date' as index, drop column named 'Date' 
time_indexed_stock_df = stock_data_df.set_index(stock_data_df['Date'])
time_indexed_stock_df = time_indexed_stock_df.drop(['Date'], axis=1)
time_indexed_stock_df.head()

,SPX,VIX_1,VIX_2,VIX_3,VIX_4,VIX_5,VIX_6,VIX_7,VIX_8,stdev,1/stdev,SPX_VOLUME,SPX_5day,SPX_tom,A_Delta
Date,,,,,,,,,,,,,,,
2009-12-31,1115.10,22.95,25.80,26.00,26.30,26.35,26.30,26.40,26.35,1.172737,0.852706,636905,NaN,1132.98,17.88
2010-01-01,1132.98,22.10,24.85,25.25,25.75,25.75,25.75,25.85,25.95,1.287977,0.776412,636905,NaN,1132.98,0.00
2010-01-04,1132.98,22.10,24.85,25.25,25.75,25.75,25.75,25.85,25.95,1.287977,0.776412,1282633,NaN,1136.52,3.54
2010-01-05,1136.52,21.55,24.45,24.85,25.45,25.50,25.50,25.55,25.70,1.386655,0.721160,1368386,NaN,1137.14,0.62
2010-01-06,1137.14,20.80,23.55,24.20,24.95,25.05,25.05,25.10,25.30,1.511385,0.661645,1252015,1130.944,1141.69,4.55


In [6]:
#Drop NaN rows
spx_5day_df = time_indexed_stock_df.dropna()
spx_5day_df.head()

,SPX,VIX_1,VIX_2,VIX_3,VIX_4,VIX_5,VIX_6,VIX_7,VIX_8,stdev,1/stdev,SPX_VOLUME,SPX_5day,SPX_tom,A_Delta
Date,,,,,,,,,,,,,,,
2010-01-06,1137.14,20.80,23.55,24.20,24.95,25.05,25.05,25.10,25.30,1.511385,0.661645,1252015,1130.944,1141.69,4.55
2010-01-07,1141.69,20.45,23.15,23.85,24.75,24.85,24.85,24.90,25.10,1.575425,0.634749,1553963,1136.262,1144.98,3.29
2010-01-08,1144.98,19.90,22.55,23.40,24.40,24.65,24.60,24.75,24.90,1.714317,0.583323,1508175,1138.662,1146.98,2.00
2010-01-11,1146.98,19.50,22.25,23.25,24.15,24.30,24.40,24.50,24.65,1.760479,0.568027,1444997,1141.462,1136.22,-10.76
2010-01-12,1136.22,20.25,22.80,23.65,24.25,24.45,24.45,24.60,24.75,1.513511,0.660716,2089364,1141.402,1145.68,9.46


In [7]:
output_df = spx_5day_df.filter(['SPX_tom', 'A_Delta'], axis=1)
output_df.head()

,SPX_tom,A_Delta
Date,,
2010-01-06,1141.69,4.55
2010-01-07,1144.98,3.29
2010-01-08,1146.98,2.00
2010-01-11,1136.22,-10.76
2010-01-12,1145.68,9.46


In [8]:
#Create list
#If "actual delta" > 0 (meaning tomorrow's price is higher than today's price)
#Assign "Buy" = 1, "Don't Buy" = 0
result_list = []
for i in range(0, len(output_df['A_Delta'])):
    if output_df['A_Delta'][i] > 0:
        result_list.append(1)
    else:
        result_list.append(0)

print(result_list[:10])

[1, 1, 1, 0, 1, 1, 0, 1, 0, 0]


In [9]:
#Create Numpy array of list and prepare for "y-train" "y-test"

y_train_data = np.asarray(result_list[:2048])
y_test_data = np.asarray(result_list[2048:])
print(y_train_data.shape)
#Reshape array for later ML model
# y_train = y_train_data.reshape(-1, 1)
# y_test = y_test_data.reshape(-1,1)
y_train = y_train_data
y_test = y_test_data

print(y_train.shape)
print(y_test.shape)
y_train


(2048,)
(2048,)
(475,)


array([1, 1, 1, ..., 0, 0, 1])

In [10]:
output_df['Act_Result'] = result_list
output_df.head()

,SPX_tom,A_Delta,Act_Result
Date,,,
2010-01-06,1141.69,4.55,1
2010-01-07,1144.98,3.29,1
2010-01-08,1146.98,2.00,1
2010-01-11,1136.22,-10.76,0
2010-01-12,1145.68,9.46,1


In [11]:
spx_5day_df = spx_5day_df.drop(columns=['stdev','SPX_tom', 'A_Delta'])

In [12]:
spx_5day_df.head()

,SPX,VIX_1,VIX_2,VIX_3,VIX_4,VIX_5,VIX_6,VIX_7,VIX_8,1/stdev,SPX_VOLUME,SPX_5day
Date,,,,,,,,,,,,
2010-01-06,1137.14,20.80,23.55,24.20,24.95,25.05,25.05,25.10,25.30,0.661645,1252015,1130.944
2010-01-07,1141.69,20.45,23.15,23.85,24.75,24.85,24.85,24.90,25.10,0.634749,1553963,1136.262
2010-01-08,1144.98,19.90,22.55,23.40,24.40,24.65,24.60,24.75,24.90,0.583323,1508175,1138.662
2010-01-11,1146.98,19.50,22.25,23.25,24.15,24.30,24.40,24.50,24.65,0.568027,1444997,1141.462
2010-01-12,1136.22,20.25,22.80,23.65,24.25,24.45,24.45,24.60,24.75,0.660716,2089364,1141.402


In [13]:
X_data = spx_5day_df.to_numpy()
print("X-Dim Total:")
print(X_data.shape[0])
print(X_data.shape[1])


X-Dim Total:
2523
12


In [14]:
train_df = spx_5day_df[:2048] 
test_df = spx_5day_df[2048:]

In [15]:

train_data = train_df.to_numpy()
print("x-train Dimensions:")
print(train_data.shape[0])
print(train_data.shape[1])
print("")

test_data = test_df.to_numpy()
print("x-test Dimensions:")
print(test_data.shape[0])
print(test_data.shape[1])
print("Note: Before Reshaping")

x-train Dimensions:
2048
12

x-test Dimensions:
475
12
Note: Before Reshaping


In [ ]:
# #julia
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler(feature_range=(0,1))
# X_scaled = scaler.fit_transform(train_data)


In [16]:
x_train = train_data.reshape(train_data.shape[0], 1,train_data.shape[1])
print(x_train.shape)

(2048, 1, 12)


In [17]:
x_test = test_data.reshape(test_data.shape[0], 1, test_data.shape[1])
print(x_test.shape)
print(x_test)

(475, 1, 12)
[[[2.58484000e+03 1.16250000e+01 1.27750000e+01 ... 5.56264982e-01
   1.04453600e+06 2.58735600e+03]]

 [[2.57887000e+03 1.20500000e+01 1.26750000e+01 ... 5.60414299e-01
   1.57321900e+06 2.58500200e+03]]

 [[2.56462000e+03 1.30500000e+01 1.31750000e+01 ... 6.39414200e-01
   1.90094100e+06 2.57905000e+03]]

 ...

 [[2.93778000e+03 1.80750000e+01 1.86750000e+01 ... 3.28027099e+00
   1.25283400e+06 2.92027200e+03]]

 [[2.97600000e+03 1.72750000e+01 1.82250000e+01 ... 2.07013067e+00
   1.66999900e+06 2.93055600e+03]]

 [[2.97871000e+03 1.65250000e+01 1.77250000e+01 ... 1.49434851e+00
   1.18228700e+06 2.94100600e+03]]]


In [18]:
from tensorflow.keras.models import Sequential
model = Sequential()

In [19]:
from tensorflow.keras.layers import LSTM

model.add(LSTM(32, input_shape=(x_train.shape[1], x_train.shape[2])))


W0928 12:40:51.068307 15840 deprecation.py:506] From C:\Users\blake\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [20]:
from tensorflow.keras.layers import Dense, Activation
model.add(Dense(1, activation='sigmoid'))
#model.add(Activation('sigmoid'))

In [21]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32)                5760      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 5,793
Trainable params: 5,793
Non-trainable params: 0
_________________________________________________________________
None


In [22]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

W0928 12:40:51.743604 15840 deprecation.py:323] From C:\Users\blake\Anaconda3\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [23]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32)                5760      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 5,793
Trainable params: 5,793
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
# # Fit (train) the model
# model.fit(
#     x_train,
#     y_train_categorical,
#     epochs=1000,
#     shuffle=True,
#     verbose=2
# )

In [25]:
model.fit(x_train, y_train, shuffle=False, epochs=100, batch_size=16)

Epoch 1/100
2048/2048 [==============================] - 2s 1ms/sample - loss: 0.6921 - acc: 0.5273
Epoch 2/100
2048/2048 [==============================] - 0s 244us/sample - loss: 0.6920 - acc: 0.5273
Epoch 3/100
2048/2048 [==============================] - 1s 252us/sample - loss: 0.6921 - acc: 0.5273
Epoch 4/100
2048/2048 [==============================] - 1s 361us/sample - loss: 0.6921 - acc: 0.5273
Epoch 5/100
2048/2048 [==============================] - 1s 394us/sample - loss: 0.6921 - acc: 0.5273
Epoch 6/100
2048/2048 [==============================] - 1s 248us/sample - loss: 0.6921 - acc: 0.5273
Epoch 7/100
2048/2048 [==============================] - 1s 248us/sample - loss: 0.6921 - acc: 0.5273
Epoch 8/100
2048/2048 [==============================] - 1s 254us/sample - loss: 0.6921 - acc: 0.5273
Epoch 9/100
2048/2048 [==============================] - 1s 250us/sample - loss: 0.6921 - acc: 0.5273
Epoch 10/100
2048/2048 [==============================] - 1s 249us/sample - loss: 0.

2048/2048 [==============================] - 1s 250us/sample - loss: 0.6921 - acc: 0.5273
Epoch 81/100
2048/2048 [==============================] - 0s 228us/sample - loss: 0.6921 - acc: 0.5273
Epoch 82/100
2048/2048 [==============================] - 0s 236us/sample - loss: 0.6921 - acc: 0.5273
Epoch 83/100
2048/2048 [==============================] - 0s 225us/sample - loss: 0.6921 - acc: 0.5273
Epoch 84/100
2048/2048 [==============================] - 0s 230us/sample - loss: 0.6921 - acc: 0.5273
Epoch 85/100
2048/2048 [==============================] - 0s 223us/sample - loss: 0.6921 - acc: 0.5273
Epoch 86/100
2048/2048 [==============================] - 0s 230us/sample - loss: 0.6921 - acc: 0.5273
Epoch 87/100
2048/2048 [==============================] - 0s 227us/sample - loss: 0.6921 - acc: 0.5273
Epoch 88/100
2048/2048 [==============================] - 0s 227us/sample - loss: 0.6921 - acc: 0.5273
Epoch 89/100
2048/2048 [==============================] - 0s 225us/sample - loss: 0.69

In [26]:
# history = model.fit(X, y, batch_size=1, epochs=100, verbose=0)

In [27]:
predictions = model.predict_classes(X, verbose=0)

NameError: name 'X' is not defined